In [1]:
from twython import Twython
import requests
from bs4 import BeautifulSoup
from time import sleep
import re
import datetime
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.utils import formataddr
import pandas as pd
import sys
import urllib

In [2]:
api_key = "WzY9J0qmLF7FIVbadabClKuWvTzCUO"
api_secret = "KzxehGabcMppkV7wBr7WrH40xz6iUnlcH01mCEWZ96Kn3PRRq3cH"
access_token = "745130381108617217-1fyc7HddbNOjaMCZj8d2SKFaAWKV1ok3nT"
access_token_secret = "F3qKtOnmp95Vk2OqGbsgSrr8T9IBeESV0rAhbEWcygTvCID"

twitter = Twython(api_key, "KzxehGcMpapkV7wBr7WrH40xz6iUnlcH01mCEWZ96Kn3PRRq3cH", access_token, access_token_secret)

In [3]:
loginid = '***'
loginpw = '***'

sender = '***@gmail.com'
reciplist = ['***@gmail.com', '***@gmail.com']

In [4]:
oldid = [twitter.get_user_timeline(screen_name='@KMA_earthquake', count=1)[0]['id']]
log_list = []

In [ ]:
while True:
    try:
        new = twitter.get_user_timeline(screen_name='@KMA_earthquake', count=1)[0]

        if new['id'] != oldid[-1]:
            sleep(180)
            imgpath = ''
            if new['text'].startswith('[지진정보]'):
                quakeinfo = new['text'].split(' ')
                quaketext = quakeinfo[3] + ' ' + quakeinfo[4] + ' ' + quakeinfo[-4] + ' ' + quakeinfo[-3] + ' ' + quakeinfo[-2] + ' ' + quakeinfo[1] + ' ' +  quakeinfo[2]
            elif new['text'].startswith('[지진속보]'):
                comp = re.compile(': .*\n')
                quakeinfo = comp.findall(new['text'])
                quaketext = quakeinfo[2][2:-1] + '규모' + quakeinfo[0][2:-1] + ' 지진발생 ' + quakeinfo[1][2:-1]
            elif new['text'].startswith('[국외지진정보]'):
                quakeinfo = new['text'].split(' ')
                new = twitter.get_user_timeline(screen_name='@KMA_earthquake', count=1)[0]
                quakeinfo = new['text'].split(' ')
                quaketext = quakeinfo[6] + ' ' + quakeinfo[7] + ' ' + quakeinfo[8] + ' ' + quakeinfo[9] + ' 지역 ' + quakeinfo[11] + quakeinfo[12][0:3] + ' 지진발생 ' + quakeinfo[1][0:4] + '-' + quakeinfo[2][0:2] + '-' + quakeinfo[3][0:2] + ' ' + quakeinfo[4][0:2] + ':' + quakeinfo[5][0:2]

            url = 'http://www.weather.go.kr/weather/earthquake_volcano/report.jsp'
            soup = BeautifulSoup(requests.get(url).text, 'lxml')
            img = soup.select_one('.img_center_B20').find('img').get('src')
            imgurl = 'http://www.kma.go.kr' + img

            imgname = imgurl.split('/')[-1] 
            imgpath = 'quakeimg/' + imgname
            urllib.request.urlretrieve(imgurl, imgpath)

            msgRoot = MIMEMultipart('related')
            msgRoot['Subject'] = Header(quaketext ,'utf8')#subject
            msgRoot['From'] = formataddr((str(Header(u'M.Robo', 'utf-8')), sender))
            msgRoot['To'] = ','.join(reciplist)

            msgAlternative = MIMEMultipart('alternative')
            msgRoot.attach(msgAlternative)

            imgopen = open(imgpath, 'rb')
            msgimage = MIMEImage(imgopen.read())
            imgopen.close()
            msgimage.add_header('Content-ID', '<%s>' % (imgpath))
            msgRoot.attach(msgimage)

            text = new['text']
            text += '\n\n\n <img src="cid:%s">' % (imgpath)

            msgText = MIMEText('%s' % (text), 'html')
            msgAlternative.attach(msgText)
            s = smtplib.SMTP_SSL('smtp.gmail.com',465)
            s.login(loginid, loginpw) 
            s.sendmail(sender, reciplist, msgRoot.as_string())
            s.quit()

            oldid.append(new['id'])
            print('mailing')
            sleep(60)

        else:
            sleep(60)

    except:
        try:
            e = sys.exc_info()

            msgRoot = MIMEMultipart('related')
            msgRoot['Subject'] = Header('QuakeBot stopped' ,'utf8')#subject
            msgRoot['From'] = formataddr((str(Header(u'M.Robo', 'utf-8')), sender))
            msgRoot['To'] = ','.join('***@daum.net')

            msgAlternative = MIMEMultipart('alternative')
            msgRoot.attach(msgAlternative)

            msgText = MIMEText(str(e), 'html')
            msgAlternative.attach(msgText)
            s = smtplib.SMTP_SSL('smtp.gmail.com',465)
            s.login(loginid, loginpw) 
            s.sendmail(sender, '***@daum.net', msgRoot.as_string())
            s.quit()
            pass
        
        except:
            e = sys.exc_info()
            nowtime = datetime.datetime.now()
            nowtime = nowtime.strftime('%Y-%m-%d %H:%M:%S')
            log_list.append({'time' : nowtime, 'error' : e})
            log_time = open('logfile/지진봇log_time.txt', 'w')
            print(log_list)
            log_time.close()
            sleep(60)
            pass